In [1]:
import numpy as np
import pandas as pd
import statistics
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [3]:
train_col_names = ['qid', 'question_text', 'target']
train_data = pd.read_csv('train.csv', names=train_col_names, skiprows=1)
train_data.head()

,qid,question_text,target
0,6f47b0f60633c2056455,"How can I reply to this comment, ""India is poo...",0
1,d49b3966070b27bf07fc,What did they use for transportation in Ancien...,0
2,6d5faa49380557c8ca7b,What are the most important provisions of Obam...,0
3,cebea75faa47388edcf5,At what age do most Finns master English today?,0
4,2a7b76a679cadb0a016e,What is cheapest place to live in India for on...,0


In [4]:
train_data.tail()

,qid,question_text,target
783668,8462d2ead608fe1dcf9b,What causes a lump on the back of your head?,0
783669,12d225a4e28317c02151,What is standalone in simple way?,0
783670,03edc045cadbb11558dd,How do you prevent having low potassium while ...,0
783671,c6fb7dc8d0b34160dbe4,What is it like to meet the band Imagine Dragons?,0
783672,6a58b525dc9ccaf25b72,"How can I find an intelligent, well reasoned p...",1


In [5]:
train_ids = train_data.qid.tolist()
train_questions = train_data.question_text.tolist()

In [6]:
train_labels = train_data.target.values.tolist()

In [7]:
print(train_ids[:5])
print(train_questions[:5])
print(train_labels[:5])

[&#39;6f47b0f60633c2056455&#39;, &#39;d49b3966070b27bf07fc&#39;, &#39;6d5faa49380557c8ca7b&#39;, &#39;cebea75faa47388edcf5&#39;, &#39;2a7b76a679cadb0a016e&#39;]
[&#39;How can I reply to this comment, &quot;India is poor. It is a fact. I don\&#39;t understand the unnecessary criticism of Snapchat CEO\&#39;s statement&quot;?&#39;, &#39;What did they use for transportation in Ancient India?&#39;, &#39;What are the most important provisions of Obamacare?&#39;, &#39;At what age do most Finns master English today?&#39;, &#39;What is cheapest place to live in India for one month which best suits for tourism also?&#39;]
[0, 0, 0, 0, 0]


In [8]:
test_col_names = ['qid', 'question_text']
test_data = pd.read_csv('test.csv', names=test_col_names, skiprows=1)
test_data.head()

,qid,question_text
0,f56a9a31974dc66186e8,Is it a good idea to go through a phlebotomy c...
1,d957c3758060f45da303,How can I fix a lead into a camlin compass to ...
2,ad822d5abaedb9e247b9,How many animes are there in world?
3,4e979c23eeb6a4bd1f2e,How do I tell my family I cut?
4,333cc031262566b8da49,How do I save down my bitcoin image address fr...


In [9]:
test_data.tail()

,qid,question_text
522444,e8e6aa5226f36c27fe41,"In terms of absolute passion and leadership, w..."
522445,015fd068afcb9d0b4007,How do I make my orthodox Indian parents accep...
522446,9f0ef49eff6a3ff9e735,How can air force officers go to ranger school?
522447,d6b02f52f76dc4c22afd,Why does my brothers cow kick me all the time?
522448,132ef601b08de269aee9,How was Japan's national animal chosen?


In [10]:
test_ids = test_data.qid.tolist()
test_questions = test_data.question_text.tolist()

In [11]:
print(test_ids[:5])
print(test_questions[:5])

[&#39;f56a9a31974dc66186e8&#39;, &#39;d957c3758060f45da303&#39;, &#39;ad822d5abaedb9e247b9&#39;, &#39;4e979c23eeb6a4bd1f2e&#39;, &#39;333cc031262566b8da49&#39;]
[&#39;Is it a good idea to go through a phlebotomy class before going into nursing school next year? Will it give me any advantage at all?&#39;, &#39;How can I fix a lead into a camlin compass to draw arcs, circles in engineering graphics?&#39;, &#39;How many animes are there in world?&#39;, &#39;How do I tell my family I cut?&#39;, &#39;How do I save down my bitcoin image address from coinbase?&#39;]


In [12]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

train_questions_new = []
test_questions_new = []

for sample_question in train_questions:
  sample_question_temp = sample_question.lower()
  train_questions_new.append(tokenizer.tokenize(sample_question_temp))

for sample_question in test_questions:
  sample_question_temp = sample_question.lower()
  test_questions_new.append(tokenizer.tokenize(sample_question_temp))

In [13]:
print(train_questions_new[:5])
print(test_questions_new[:5])

[[&#39;how&#39;, &#39;can&#39;, &#39;i&#39;, &#39;reply&#39;, &#39;to&#39;, &#39;this&#39;, &#39;comment&#39;, &#39;india&#39;, &#39;is&#39;, &#39;poor&#39;, &#39;it&#39;, &#39;is&#39;, &#39;a&#39;, &#39;fact&#39;, &#39;i&#39;, &#39;don&#39;, &#39;t&#39;, &#39;understand&#39;, &#39;the&#39;, &#39;unnecessary&#39;, &#39;criticism&#39;, &#39;of&#39;, &#39;snapchat&#39;, &#39;ceo&#39;, &#39;s&#39;, &#39;statement&#39;], [&#39;what&#39;, &#39;did&#39;, &#39;they&#39;, &#39;use&#39;, &#39;for&#39;, &#39;transportation&#39;, &#39;in&#39;, &#39;ancient&#39;, &#39;india&#39;], [&#39;what&#39;, &#39;are&#39;, &#39;the&#39;, &#39;most&#39;, &#39;important&#39;, &#39;provisions&#39;, &#39;of&#39;, &#39;obamacare&#39;], [&#39;at&#39;, &#39;what&#39;, &#39;age&#39;, &#39;do&#39;, &#39;most&#39;, &#39;finns&#39;, &#39;master&#39;, &#39;english&#39;, &#39;today&#39;], [&#39;what&#39;, &#39;is&#39;, &#39;cheapest&#39;, &#39;place&#39;, &#39;to&#39;, &#39;live&#39;, &#39;in&#39;, &#39;india&#39;, &#39;

In [14]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
print(stop_words)
print(len(stop_words))

{&#39;his&#39;, &#39;i&#39;, &#39;re&#39;, &#39;be&#39;, &quot;that&#39;ll&quot;, &#39;ain&#39;, &quot;you&#39;re&quot;, &#39;herself&#39;, &#39;too&#39;, &#39;theirs&#39;, &#39;it&#39;, &#39;out&#39;, &quot;it&#39;s&quot;, &#39;when&#39;, &quot;hasn&#39;t&quot;, &#39;if&#39;, &#39;have&#39;, &#39;each&#39;, &quot;mustn&#39;t&quot;, &#39;am&#39;, &quot;didn&#39;t&quot;, &#39;as&#39;, &#39;to&#39;, &#39;he&#39;, &#39;aren&#39;, &#39;she&#39;, &#39;do&#39;, &quot;isn&#39;t&quot;, &#39;their&#39;, &#39;weren&#39;, &#39;now&#39;, &#39;hasn&#39;, &#39;some&#39;, &#39;s&#39;, &#39;all&#39;, &#39;yours&#39;, &quot;doesn&#39;t&quot;, &#39;were&#39;, &#39;what&#39;, &#39;itself&#39;, &#39;after&#39;, &#39;can&#39;, &#39;them&#39;, &#39;hers&#39;, &quot;weren&#39;t&quot;, &#39;doing&#39;, &#39;yourselves&#39;, &#39;which&#39;, &#39;or&#39;, &#39;had&#39;, &#39;nor&#39;, &#39;needn&#39;, &#39;himself&#39;, &#39;during&#39;, &quot;you&#39;ve&quot;, &#39;an&#39;, &#39;shan&#39;, &#39;did&#39;, &#39

In [15]:
word_dict = {}
word_index = 0

In [29]:
def clean_stemm_stopwords(input_list):
  filtered_op = []

  for words_in_temp_sample in input_list:
    temp_sample = ''
    for word in words_in_temp_sample:
      if not word in stop_words and not word in temp_sample:
        temp_word = ps.stem(word)
        temp_sample += (temp_word + ' ')
        if temp_word not in word_dict.keys():
          word_dict[temp_word] = len(word_dict)
          
    temp_sample = temp_sample[:-1]
    filtered_op.append(temp_sample)

  return filtered_op

In [30]:
train_questions_filtered = clean_stemm_stopwords(train_questions_new)
test_questions_filtered = clean_stemm_stopwords(test_questions_new)

In [31]:
freq = np.asarray([np.zeros(len(word_dict)), np.zeros(len(word_dict))])

In [32]:
for x,y in zip(train_questions_filtered, train_labels):
    for w in x.split():
        freq[y, word_dict[w]] += 1

In [36]:
def feature_extraction(l):
    x = np.asarray([np.ones(len(l)), np.zeros(len(l)), np.zeros(len(l))])
    for i, s in enumerate(l):
        for w in s.split():
            x[1, i] += freq[0, word_dict[w]]
            x[2, i] += freq[1, word_dict[w]]
    return x

In [37]:
xtrain = feature_extraction(train_questions_filtered)
xsubmission = feature_extraction(test_questions_filtered)

In [46]:
xtrain = xtrain.transpose()

In [43]:
train_labels = np.asarray(train_labels)

In [44]:
train_labels.shape

(783673,)

In [103]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [47]:
xtrain, xtest, ytrain, ytest = train_test_split(xtrain, train_labels, test_size = 0.1)

In [48]:
stc = StandardScaler()
xtrain = stc.fit_transform(xtrain)
xtest = stc.transform(xtest)


In [95]:
xsubmission = stc.transform(xsubmission.transpose())

In [88]:
lg = LogisticRegression(random_state = 12, max_iter = 1000, class_weight = {0:0.1, 1:0.9})

In [89]:
lg.fit(xtrain, ytrain)

LogisticRegression(C=1.0, class_weight={0: 0.1, 1: 0.9}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class=&#39;auto&#39;, n_jobs=None, penalty=&#39;l2&#39;,
                   random_state=12, solver=&#39;lbfgs&#39;, tol=0.0001, verbose=0,
                   warm_start=False)

In [109]:
class_weights = np.asarray([0.94 if i == 1 else 0.06 for i in ytrain])

In [110]:
xg = XGBClassifier()
xg.fit(xtrain, ytrain, sample_weight = class_weights)

XGBClassifier(base_score=0.5, booster=&#39;gbtree&#39;, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type=&#39;gain&#39;, interaction_constraints=&#39;&#39;,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=&#39;()&#39;,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective=&#39;binary:logistic&#39;, random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method=&#39;exact&#39;, validate_parameters=1, verbosity=None)

In [101]:
def measure(x, y, model):
    y_pred = model.predict(x)
    return accuracy_score(y_pred, y), f1_score(y_pred, y)

In [111]:
measure(xtrain, ytrain, xg)

(0.790872034084545, 0.3354493969443981)

In [112]:
measure(xtest, ytest, xg)

(0.7896718048182931, 0.32829373650107996)

In [107]:
measure(xtrain, ytrain, lg)

(0.8999581741232516, 0.41106752357900006)

In [108]:
measure(xtrain, ytrain, lg)

(0.8999581741232516, 0.41106752357900006)

In [93]:
sub_df = pd.read_csv('sample_submission.csv')

In [97]:
sub_df['target'] = lg.predict(xsubmission)

In [99]:
sub_df.to_csv('sub3.csv', index = False)